In [ ]:
import json
import time
import shutil
import requests

In [ ]:
# bounding box using WRFHydro's Lambert Conformal Conic variant projection
llat=571663.2008999996
ulat=591703.6239999996
llon=2011337.7544
ulon=2029842.1457

In [ ]:
# submit a job via bounding box
submit_url = f'https://subset.cuahsi.org/nwm/v2_0/subset?llat={llat}&llon={llon}&ulat={ulat}&ulon={ulon}&hucs=[]'
res = requests.get(submit_url)
assert res.status_code == 200

In [ ]:
# grab the job identifier
uid = res.url.split('jobid=')[-1]

In [ ]:
# query job status
status_url = f'https://subset.cuahsi.org/jobs/{uid}'

attempt = 0
max_attempts = 100
while attempt < max_attempts:
    res = requests.get(status_url)
    status = json.loads(res.text)['status']
    if status == 'finished':
        break
    attempt += 1
    time.sleep(5)
    

In [ ]:
# download the result
dl_url = f'https://subset.cuahsi.org/download-gzip/{uid}'
#dl_url = f'https://subset.cuahsi.org/download-zip/{uid}'

local_filename = f'{uid}.tar.gz'
with requests.get(dl_url, stream=True) as r:
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [ ]:
!tar -xzvf "$uid".tar.gz